# Test McpExecutor Run Method

This notebook tests the `run` method of the `McpExecutor` class from `autocoder.common.mcp_tools`.

In [1]:
import os
import sys
import json
import asyncio
import tempfile
from pathlib import Path

from autocoder.common.mcp_hub import McpHub, McpServer, McpTool, McpResource, McpResourceTemplate
from autocoder.common.mcp_tools import McpExecutor
import byzerllm

In [2]:
# Helper function for async tests
async def run_test(test_func):
    """Helper to run async test functions"""
    try:
        await test_func()
        print("✅ Test passed")
    except AssertionError as e:
        print(f"❌ Test failed: {str(e)}")
    except Exception as e:
        print(f"❌ Test error: {str(e)}")

In [6]:
import json
async def test_mcp_executor_run():
    """Test McpExecutor run method"""
    # Create settings file with filesystem server config
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        settings = {
            "mcpServers": {
                "filesystem": {
                    "command": "npx",
                    "args": [
                        "-y",
                        "@modelcontextprotocol/server-filesystem",
                        "/Users/allwefantasy/projects/tests"                        
                    ]
                }
            }
        }
        json.dump(settings, f)
        settings_path = f.name
    
    try:
        # Initialize hub and executor
        hub = McpHub()
        # await hub.initialize()
        
        llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
        executor = McpExecutor(hub, llm)
        
        # Create test conversation
        conversations = [{
            "role": "user",
            "content": "List the contents of /Users/allwefantasy/projects/tests"
        }]
        
        # Run the executor
        final_conversations,results = await executor.run(conversations)        
        # print(json.dumps(final_conversations, indent=2,ensure_ascii=False))
        print(results)
               
        
    finally:
        # Cleanup
        # await hub.shutdown()
        os.unlink(settings_path)

In [8]:
# Run the test
print("Testing McpExecutor run method:")
await test_mcp_executor_run()

2025-01-02 22:33:54.602 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2025-01-02 22:33:54,638	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2025-01-02 22:33:54,638	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


Testing McpExecutor run method:
[CallToolResult(content=[TextContent(type='text', text='[FILE] a.txt')], isError=False)]
